This script will batch generate COGs from a directory of GeoTIFFs. It does not affect the bands or color profile. The output COG may be black.

In [4]:
import os
import subprocess

def reproject_and_convert_to_cog(geotiff_path, temp_reprojected_path, cog_path, target_srs="EPSG:4326"):
    """
    Reprojects a GeoTIFF to the target CRS and then converts it to a Cloud Optimized GeoTIFF (COG).
    
    Parameters:
        geotiff_path (str): Path to the input GeoTIFF file.
        temp_reprojected_path (str): Path to save the temporary reprojected GeoTIFF.
        cog_path (str): Path to save the final COG.
        target_srs (str): Target spatial reference system (e.g., "EPSG:4326").
    """
    try:
        # Step 1: Reproject the GeoTIFF using gdalwarp
        subprocess.run([
            "gdalwarp",
            "-t_srs", target_srs,          # Target spatial reference system
            "-of", "GTiff",                # Output format: GeoTIFF
            geotiff_path,
            temp_reprojected_path
        ], check=True)
        print(f"Reprojected GeoTIFF saved at {temp_reprojected_path}")
        
        # Step 2: Convert the reprojected GeoTIFF to COG using gdal_translate
        subprocess.run([
            "gdal_translate",
            "-of", "COG",                  # Output format: COG
            "-co", "COMPRESS=LZW",         # Compression option for smaller file size
            temp_reprojected_path,
            cog_path
        ], check=True)
        print(f"COG created at {cog_path}")
        
        # Clean up the temporary reprojected file
        os.remove(temp_reprojected_path)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {geotiff_path}: {e}")

def create_cogs_with_reprojection(geotiff_dir, cog_dir, target_srs="EPSG:4326"):
    """
    Scans a directory for GeoTIFF files, reprojects them, and generates COGs for each.
    
    Parameters:
        geotiff_dir (str): Directory containing GeoTIFF files (including subdirectories).
        cog_dir (str): Directory to save the generated COGs.
        target_srs (str): Target spatial reference system (e.g., "EPSG:3857").
    """
    # Ensure the COG output directory exists
    os.makedirs(cog_dir, exist_ok=True)
    
    # Gather all GeoTIFF files recursively
    geotiff_files = []
    for root, dirs, files in os.walk(geotiff_dir):
        for file in files:
            if file.endswith(".tif"):
                geotiff_files.append(os.path.join(root, file))
    
    print(f"Found {len(geotiff_files)} GeoTIFF files to process.")

    # Process each GeoTIFF file
    for geotiff_path in geotiff_files:
        # Define paths for the temporary reprojected file and the final COG
        temp_reprojected_name = f"{os.path.splitext(os.path.basename(geotiff_path))[0]}_reprojected.tif"
        temp_reprojected_path = os.path.join(cog_dir, temp_reprojected_name)
        
        cog_name = f"{os.path.splitext(os.path.basename(geotiff_path))[0]}_cog.tif"
        cog_path = os.path.join(cog_dir, cog_name)
        
        # Reproject and convert to COG
        reproject_and_convert_to_cog(geotiff_path, temp_reprojected_path, cog_path, target_srs)

# Usage
geotiff_directory = "indiana"
cog_directory = "indiana_cogs"
target_crs = "EPSG:3857"  # Reproject

create_cogs_with_reprojection(geotiff_directory, cog_directory, target_crs)



Found 16 GeoTIFF files to process.
Creating output file that is 11738P x 13606L.
Using internal nodata values (e.g. -128) for image indiana/Land_Cover_2011/LAND_COVER_2011_USGS_IN.tif.
Copying nodata values from source indiana/Land_Cover_2011/LAND_COVER_2011_USGS_IN.tif to destination indiana_cogs/LAND_COVER_2011_USGS_IN_reprojected.tif.
Processing indiana/Land_Cover_2011/LAND_COVER_2011_USGS_IN.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Reprojected GeoTIFF saved at indiana_cogs/LAND_COVER_2011_USGS_IN_reprojected.tif
Input file size is 11738, 13606
0...10...20...30...40...50...60...70...80...90...100 - done.
COG created at indiana_cogs/LAND_COVER_2011_USGS_IN_cog.tif
Creating output file that is 11738P x 13606L.
Using internal nodata values (e.g. -128) for image indiana/Land_Cover_Impervious_Surfaces_2006/IMPERVIOUS_SURFACE_2006_USGS_IN.tif.
Copying nodata values from source indiana/Land_Cover_Impervious_Surfaces_2006/IMPERVIOUS_SURFACE_2006_USGS_IN.tif t